In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
def load_or_save_model():
    model_path = "esrgan_model"  

   
    if os.path.exists(model_path):
        print("Model already exists, loading from", model_path)
        model = tf.saved_model.load(model_path)
    else:
        print("Model not found, downloading and saving the model...")
        model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")  # Load ESRGAN model
        tf.saved_model.save(model, model_path)  
    return model

model = load_or_save_model()

Model not found, downloading and saving the model...
INFO:tensorflow:Assets written to: esrgan_model/assets


INFO:tensorflow:Assets written to: esrgan_model/assets


In [3]:
# Function to preprocess video frame for model input
def preprocess_frame(frame):
    frame = cv2.resize(frame, (640, 480))  # Resize to SD resolution
    frame = tf.expand_dims(frame, 0)  
    frame = tf.cast(frame, tf.float32)
    return frame

In [4]:
# Function to postprocess video frame after model output
def postprocess_frame(frame):
    frame = tf.squeeze(frame)
    frame = tf.clip_by_value(frame, 0, 255)  # Clip pixel values
    frame = tf.cast(frame, tf.uint8)  # Cast to uint8
    return frame.numpy()

In [5]:
# Function to convert SD video to HD using diffusion model
def convert_video(input_video_path, output_video_path, model):
    input_video = cv2.VideoCapture(input_video_path)
    if not input_video.isOpened():
        print("Error: Unable to open input video file.")
        return

    fps = input_video.get(cv2.CAP_PROP_FPS)
    width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH)) * 2
    height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (1280, 720))
    if not output_video.isOpened():
        print("Error: Unable to open output video file.")
        input_video.release()
        return

    frame_count = 0
    while True:
        ret, frame = input_video.read()
        if not ret:
            break

        frame_count += 1

        
        preprocessed_frame = preprocess_frame(frame)

        
        sr_frame = model(preprocessed_frame)

        
        sr_frame = postprocess_frame(sr_frame)

       
        output_video.write(sr_frame)

    # Release video objects
    input_video.release()
    output_video.release()
    cv2.destroyAllWindows()

    print("Video conversion completed. Total frames processed:", frame_count)


In [6]:
# #main

# def main():
#     input_video_path = "file.mp4"
#     output_video_path = "output_video_jsut.mp4"

#     # Convert SD video to HD
#     convert_video(input_video_path, output_video_path, model)

# if __name__ == "__main__":
#     main()